In [1]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [2]:
ak = pd.read_csv('../datasets/alaska_single_engine.csv', low_memory=False)

In [3]:
ak.dtypes

Unnamed: 0               int64
NtsbNo                  object
EventType               object
Mkey                     int64
EventDate               object
City                    object
State                   object
Country                 object
ReportNo                object
N                       object
HasSafetyRec              bool
ReportType              object
OriginalPublishDate     object
HighestInjuryLevel      object
FatalInjuryCount         int64
SeriousInjuryCount       int64
MinorInjuryCount         int64
ProbableCause           object
EventID                float64
Latitude               float64
Longitude              float64
Make                    object
Model                   object
AirCraftCategory        object
AirportID               object
AirportName             object
AmateurBuilt            object
NumberOfEngines         object
Scheduled               object
PurposeOfFlight         object
FAR                     object
AirCraftDamage          object
WeatherC

In [4]:
ak.head()

,Unnamed: 0,NtsbNo,EventType,Mkey,EventDate,City,State,Country,ReportNo,N,HasSafetyRec,ReportType,OriginalPublishDate,HighestInjuryLevel,FatalInjuryCount,SeriousInjuryCount,MinorInjuryCount,ProbableCause,EventID,Latitude,Longitude,Make,Model,AirCraftCategory,AirportID,AirportName,AmateurBuilt,NumberOfEngines,Scheduled,PurposeOfFlight,FAR,AirCraftDamage,WeatherCondition,Operator,ReportStatus,RepGenFlag,DocketUrl,DocketPublishDate,Unnamed: 37
0,80,ANC23LA086,ACC,193153,2023-09-24T08:30:00Z,Trimble River / Skwentna,Alaska,United States,NaN,N2586R,False,DirectorBrief,NaN,NaN,0,0,0,NaN,NaN,61.775160,-152.15263,CESSNA,182K,AIR,NaN,Trimble River,FALSE,1,NaN,PERS,091,Substantial,NaN,NaN,In work,False,NaN,NaN,NaN
1,94,ANC23LA084,ACC,193128,2023-09-20T12:00:00Z,Bethel,Alaska,United States,NaN,N8192D,False,DirectorBrief,NaN,NaN,0,0,0,NaN,NaN,60.805019,-161.78648,PIPER,PA-18-150,AIR,NaN,NaN,FALSE,1,NaN,PERS,091,Substantial,NaN,NaN,In work,False,NaN,NaN,NaN
2,100,ANC23LA080,ACC,193097,2023-09-18T13:00:00Z,Homer,Alaska,United States,NaN,N7558H,False,DirectorBrief,NaN,NaN,0,0,0,NaN,NaN,59.646929,-151.49323,CESSNA,A185F,AIR,5BL,HOMER-BELUGA LAKE,FALSE,1,NaN,BUS,091,Substantial,VMC,Adventure Airways,In work,False,NaN,NaN,NaN
3,112,ANC23LA082,ACC,193105,2023-09-16T16:50:00Z,Beaver Creek,Alaska,United States,NaN,N713C,False,DirectorBrief,NaN,NaN,0,0,0,NaN,NaN,64.267579,-147.68704,HELIO,H-295,AIR,NaN,NaN,FALSE,1,NSCH,BUS,135,Substantial,NaN,WRIGHT AIR SERVICE INC,In work,False,NaN,NaN,NaN
4,113,ANC23LA078,ACC,193088,2023-09-16T15:00:00Z,NENANA,Alaska,United States,NaN,N907W,False,DirectorBrief,NaN,NaN,0,0,0,NaN,NaN,64.650753,-149.83639,RHODES STEVEN D,SR3500,AIR,NaN,NaN,TRUE,1,NaN,PERS,091,Substantial,VMC,NaN,In work,False,NaN,NaN,NaN


In [5]:
ak.isna().sum()

Unnamed: 0                0
NtsbNo                    0
EventType                 0
Mkey                      0
EventDate                 0
City                      0
State                     0
Country                   0
ReportNo               5645
N                         0
HasSafetyRec              0
ReportType               22
OriginalPublishDate     885
HighestInjuryLevel     4044
FatalInjuryCount          0
SeriousInjuryCount        0
MinorInjuryCount          0
ProbableCause          1452
EventID                5659
Latitude                  0
Longitude                 0
Make                      0
Model                     1
AirCraftCategory          0
AirportID              3510
AirportName            3196
AmateurBuilt              0
NumberOfEngines           0
Scheduled              4342
PurposeOfFlight         414
FAR                       0
AirCraftDamage           52
WeatherCondition         20
Operator               2842
ReportStatus              0
RepGenFlag          

In [6]:
ak_clean = ak

In [7]:
ak_clean.fillna({'HighestInjuryLevel': 'None Reported',
                 'AirCraftDamage': 'None Reported',
                 }, inplace=True)

In [8]:
ak_clean.fillna({'WeatherCondition': 'Unknown',
                 'Scheduled': 'Unknown',
                 'Operator': 'Unknown',
                 'AirportID': 'Unknown',
                 'AirportName': 'Unknown',
                 'PurposeOfFlight': 'Unknown',
                 'ReportType': 'Unknown',
                 'ProbableCause': 'Unknown',
                 'Model': 'Unknown'}, inplace=True)

In [9]:
# Remap
ak_clean['AmateurBuilt'] = ak_clean['AmateurBuilt'].replace({'FALSE,FALSE': 0, 'FALSE': 0, 'TRUE': 1})

In [10]:
ak_clean['HasSafetyRec'] = ak_clean['HasSafetyRec'].replace({'False': 0, 'True': 1}).astype(int)

In [11]:
# Convert to datetime
ak_clean['EventDate'] = pd.to_datetime(ak_clean['EventDate'])

In [12]:
ak_clean['EventYear'] = ak_clean['EventDate'].dt.year

In [13]:
ak_clean['EventMonth'] = ak_clean['EventDate'].dt.month

In [14]:
ak_clean['EventDay'] = ak_clean['EventDate'].dt.day

In [15]:
ak_clean['EventTime'] = ak_clean['EventDate'].dt.time

In [16]:
ak_clean['EventHour'] = [x.hour for x in ak_clean['EventTime']]

In [17]:
def get_season(month):
    if 3 <= month <= 5:
        return "Spring"
    elif 6 <= month <= 8:
        return "Summer"
    elif 9 <= month <= 11:
        return "Fall"
    else:
        return "Winter"

In [18]:
ak_clean['EventSeason'] = ak_clean['EventMonth'].apply(get_season)

In [19]:
# drop unnecessary columns
ak_clean = ak.drop(['Unnamed: 0', 'Unnamed: 37', 'DocketUrl', 'DocketPublishDate', 'State', 'OriginalPublishDate', 'NumberOfEngines', 'ReportNo', 'Country', 'EventID', 'EventDate', 'ReportStatus', 'RepGenFlag'], axis=1)

In [20]:
ak_clean.isna().sum()

NtsbNo                0
EventType             0
Mkey                  0
City                  0
N                     0
HasSafetyRec          0
ReportType            0
HighestInjuryLevel    0
FatalInjuryCount      0
SeriousInjuryCount    0
MinorInjuryCount      0
ProbableCause         0
Latitude              0
Longitude             0
Make                  0
Model                 0
AirCraftCategory      0
AirportID             0
AirportName           0
AmateurBuilt          0
Scheduled             0
PurposeOfFlight       0
FAR                   0
AirCraftDamage        0
WeatherCondition      0
Operator              0
EventYear             0
EventMonth            0
EventDay              0
EventTime             0
EventHour             0
EventSeason           0
dtype: int64

In [21]:
def camel_to_snake(column_name):
    result = []
    i = 0

    while i < len(column_name):
        if column_name[i].isupper():
            if i > 0 and not column_name[i - 1].isupper():
                result.append('_')
            result.append(column_name[i].lower())
        else:
            result.append(column_name[i])
        i += 1

    return ''.join(result)

In [22]:
ak_clean.columns = [camel_to_snake(column) for column in ak_clean.columns]

In [23]:
ak_clean.dtypes

ntsb_no                  object
event_type               object
mkey                      int64
city                     object
n                        object
has_safety_rec            int64
report_type              object
highest_injury_level     object
fatal_injury_count        int64
serious_injury_count      int64
minor_injury_count        int64
probable_cause           object
latitude                float64
longitude               float64
make                     object
model                    object
air_craft_category       object
airport_id               object
airport_name             object
amateur_built             int64
scheduled                object
purpose_of_flight        object
far                      object
air_craft_damage         object
weather_condition        object
operator                 object
event_year                int32
event_month               int32
event_day                 int32
event_time               object
event_hour                int64
event_se

In [24]:
ak_clean.head(15)

,ntsb_no,event_type,mkey,city,n,has_safety_rec,report_type,highest_injury_level,fatal_injury_count,serious_injury_count,minor_injury_count,probable_cause,latitude,longitude,make,model,air_craft_category,airport_id,airport_name,amateur_built,scheduled,purpose_of_flight,far,air_craft_damage,weather_condition,operator,event_year,event_month,event_day,event_time,event_hour,event_season
0,ANC23LA086,ACC,193153,Trimble River / Skwentna,N2586R,0,DirectorBrief,None Reported,0,0,0,Unknown,61.775160,-152.15263,CESSNA,182K,AIR,Unknown,Trimble River,0,Unknown,PERS,091,Substantial,Unknown,Unknown,2023,9,24,08:30:00,8,Fall
1,ANC23LA084,ACC,193128,Bethel,N8192D,0,DirectorBrief,None Reported,0,0,0,Unknown,60.805019,-161.78648,PIPER,PA-18-150,AIR,Unknown,Unknown,0,Unknown,PERS,091,Substantial,Unknown,Unknown,2023,9,20,12:00:00,12,Fall
2,ANC23LA080,ACC,193097,Homer,N7558H,0,DirectorBrief,None Reported,0,0,0,Unknown,59.646929,-151.49323,CESSNA,A185F,AIR,5BL,HOMER-BELUGA LAKE,0,Unknown,BUS,091,Substantial,VMC,Adventure Airways,2023,9,18,13:00:00,13,Fall
3,ANC23LA082,ACC,193105,Beaver Creek,N713C,0,DirectorBrief,None Reported,0,0,0,Unknown,64.267579,-147.68704,HELIO,H-295,AIR,Unknown,Unknown,0,NSCH,BUS,135,Substantial,Unknown,WRIGHT AIR SERVICE INC,2023,9,16,16:50:00,16,Fall
4,ANC23LA078,ACC,193088,NENANA,N907W,0,DirectorBrief,None Reported,0,0,0,Unknown,64.650753,-149.83639,RHODES STEVEN D,SR3500,AIR,Unknown,Unknown,1,Unknown,PERS,091,Substantial,VMC,Unknown,2023,9,16,15:00:00,15,Fall
5,ANC23LA083,ACC,193106,McGrath,N1755P,0,DirectorBrief,None Reported,0,0,0,Unknown,62.946312,-155.52845,PIPER,PA-18-150,AIR,Unknown,Unknown,0,Unknown,PERS,091,Substantial,Unknown,Unknown,2023,9,15,17:30:00,17,Fall
6,ANC23FA074,ACC,193053,St. Marys,N109T,0,DirectorBrief,Fatal,1,0,0,Unknown,62.863731,-161.77257,PIPER,PA-18-150,AIR,Unknown,Unknown,0,NSCH,Unknown,135,Substantial,VMC,NEITZ AVIATION INC,2023,9,12,19:47:00,19,Fall
7,ANC23LA077,ACC,193070,Palmer,N95250,0,DirectorBrief,None Reported,0,0,0,Unknown,61.486395,-148.34239,TAYLORCRAFT,BC12-D,AIR,Unknown,Unknown,0,Unknown,PERS,091,Substantial,Unknown,GRIGGS JULIAN JL,2023,9,12,17:30:00,17,Fall
8,ANC23LA075,ACC,193057,Healy,N34VV,0,DirectorBrief,None Reported,0,0,0,Unknown,64.160000,-148.30000,CESSNA,185,AIR,Unknown,Unknown,0,Unknown,POSI,091,Substantial,Unknown,WATTUM INVESTMENTS LLC,2023,9,12,13:15:00,13,Fall
9,ANC23LA081,ACC,193100,Juneau,N4900C,0,DirectorBrief,Serious,0,1,0,Unknown,58.596670,-135.51560,PIPER/GERBI CHARLES R,PA 18-150,AIR,Unknown,Unknown,0,Unknown,PERS,091,Substantial,Unknown,RIDER BRADLEY H,2023,9,10,16:30:00,16,Fall


In [25]:
def comma_exploder(df, columns_to_explode):
    new_rows = []

    for _, row in df.iterrows():
        if any(isinstance(row[column], str) and ',' in row[column] for column in columns_to_explode):
            values_to_split = {column: row[column].split(', ') if isinstance(row[column], str) and ',' in row[column] else [row[column]] for column in columns_to_explode}
            for i in range(len(max(values_to_split.values(), key=len))):
                new_row = row.copy()
                for column, values in values_to_split.items():
                    if i < len(values):
                        new_row[column] = values[i]
                    else:
                        new_row[column] = ''
                new_rows.append(new_row)
        else:
            new_rows.append(row)

    exploded_df = pd.DataFrame(new_rows)

    exploded_df.reset_index(drop=True, inplace=True)

    return exploded_df

In [26]:
ak_clean = comma_exploder(ak_clean, ['n', 'make', 'model', 'air_craft_category', 'purpose_of_flight', 'far', 'air_craft_damage'])

In [27]:
ak_clean.to_csv('../datasets/alaska_single_engine_clean.csv', index=False)